# Functor

A Functor is a type class and it is any data type that defines how map applies to it.

We have a Constructor C[_] and two types A and B, we want to apply functions of type C[A] => C[B], so we need adequate tranformations.

In [7]:
trait Functor[F[_]] {
    def map
}

defined trait Functor

![](images/functor2.png)

We can test it with a class like Option:

In [17]:
sealed abstract class OptionFun[+A] {
    def getValue: A = ???
    
    def isEmpty: Boolean = ???
    def map[B](f:A => B): OptionFun[B] = ???
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = ???
}

case object NoneStuff extends OptionFun[Nothing]

defined class OptionFun
defined class SomeStuff
defined object NoneStuff

In [16]:
val opt = SomeStuff(15)
println("new value " + opt.map(_.toString).getValue)

new value 15


opt: SomeStuff[Int] = SomeStuff(15)

# Applicative

Applicative is a type class and it is any data type that defines how apply applies to it.

Apply takes a functor that has a function in it and another functor and extracts that function from the first functor and then maps it over the second one.

We have a Constructor C[_] and two types A and B, we want to apply functions of type C[A] => C[B], so we need adequate tranformations.

In [5]:
trait Applicative[F[_]] extends Functor[F] {
    def pure
    def apply
    def map
}

defined trait Applicative

![](images/applicative.png)

Using our class OptionFun:

In [23]:
object OptionFun {
    def pure[A](x: A): OptionFun[A] = ???
}
sealed abstract class OptionFun[+A] {
    def getValue: A = ???
    
    def isEmpty: Boolean = ???
    def map[B](f:A => B): OptionFun[B] = ???
    def apply[B](f: OptionFun[A => B]): OptionFun[B] = ???
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = ???
}

case object NoneStuff extends OptionFun[Nothing]

defined object OptionFun
defined class OptionFun
defined class SomeStuff
defined object NoneStuff

In [24]:
val a: Int => String = _.toString

a: Int => String = ammonite.$sess.cmd23$Helper$$Lambda$2806/1150605593@4d727083

In [25]:
val functionOptionFun = OptionFun.pure(a)

functionOptionFun: OptionFun[Int => String] = SomeStuff(
  ammonite.$sess.cmd23$Helper$$Lambda$2806/1150605593@4d727083
)

In [27]:
val opt = SomeStuff(15)
println("new value " + opt.apply(functionOptionFun).getValue)

new value 15


opt: SomeStuff[Int] = SomeStuff(15)

Maybe seems like this is not usefull but it is

# Monads

Monads apply a function that returns a wrapped value to a wrapped value

![](images/monad.png)

Monad is a type class and it's a data type that implements the flatMap.

We have a Constructor C[_] and two types A and B, we want to apply functions of type C[A] => C[B], so we need adequate tranformations.

In [28]:
trait Monad[F[_]] extends Applicative[F] {
    def flatMap
}

defined trait Monad

Using our class OptionFun:

In [29]:
object OptionFun {
    def pure[A](x: A): OptionFun[A] = ???
}
sealed abstract class OptionFun[+A] {
    def getValue: A = ???
    def isEmpty: Boolean = ???
    def map[B](f:A => B): OptionFun[B] = ???
    def apply[B](f: OptionFun[A => B]): OptionFun[B] = ???
    def flatMap[B](f: A => OptionFun[B]): OptionFun[B] = ???
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = ???
}

case object NoneStuff extends OptionFun[Nothing]

defined object OptionFun
defined class OptionFun
defined class SomeStuff
defined object NoneStuff

Why is so important the flatMap method? With it you can compose functions with wrapped values. Let see and example

In [30]:
val a = SomeStuff(5)
val b = SomeStuff(6)
val c = SomeStuff(7)

a: SomeStuff[Int] = SomeStuff(5)
b: SomeStuff[Int] = SomeStuff(6)
c: SomeStuff[Int] = SomeStuff(7)

if we want to summ all the values, with flatMap and map we can:

In [31]:
a.flatMap(value => b.flatMap(value2 => c.map(value3 => value + value2 + value3)))

res30: OptionFun[Int] = SomeStuff(18)

But this is not pretty. We can do it in a readable form using for comprehension:

In [32]:
for {
    value1 <- a
    value2 <- b
    value3 <- c
} yield (value1 + value2 + value3)

res31: OptionFun[Int] = SomeStuff(18)